In [1]:
from datasets import load_dataset, load_from_disk
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [2]:

class HMMCustom:
    def __init__(self, n_components, n_observations, startprob=None, transmat=None, emissionprob=None, strategy="viterbi"):

        self.n_components_ = n_components
        self.n_observations_ = n_observations
        self.strategy = strategy

        if startprob is None:
            self.startprob_ = np.zeros(n_components)
        else:
            self.startprob_ = startprob
        if transmat is None:
            self.transmat_ = np.zeros((n_components, n_components))
        else:
            self.transmat_ = transmat
        if emissionprob is None:
            self.emissionprob_ = np.zeros((n_components, n_observations))
        else:
            self.emissionprob_ = emissionprob

    def fit(self, X, y):

        # Get start & transition & emission probs
        for idx, sentence in enumerate(X):
            for i, word in enumerate(sentence):

                # Get start prob
                if i == 0:
                    self.startprob_[y[idx][i]] += 1
                # Get transition prob
                else:
                    self.transmat_[y[idx][i], y[idx][i-1]] += 1

                # Get emission prob
                self.emissionprob_[y[idx][i], word] += 1

        # Get start & transition & emission probs
        for i in range(self.n_components_):
            self.startprob_[i] += 1
            for j in range(self.n_components_):
                self.transmat_[i, j] += 1
            for j in range(self.n_observations_):
                self.emissionprob_[i, j] += 1

        with np.errstate(divide='ignore', invalid='ignore'):
            self.startprob_ /= np.sum(self.startprob_)
            self.emissionprob_ /= ( np.sum(self.emissionprob_, axis=1).reshape(-1, 1) )
            # self.emissionprob_ /= ( np.sum(self.transmat_, axis=1).reshape(-1, 1) + self.startprob_.reshape(-1, 1) )
            self.transmat_ /= np.sum(self.transmat_, axis=1).reshape(-1, 1)
            # self.transmat_ /= np.sum(self.transmat_, axis=1)[:, np.newaxis]
        self.transmat_ = np.nan_to_num(self.transmat_)
        self.emissionprob_ = np.nan_to_num(self.emissionprob_)

        # print(self.startprob_)
        # print()
        # for x in self.transmat_:
        #     print(x)
        # print()
        # for x in self.emissionprob_:
        #     print(x)
        # print()

    def _greedy(self, X):
        log_likelihood, hidden_states = 0, []
        prev_state = None

        for i, word in enumerate(X):
            score = -1
            if i == 0:
                for state in range(self.n_components_):
                    prob = self.startprob_[state] * self.emissionprob_[state][word]
                    if prob > score:
                        score = prob
                        prev_state = state
            else:
                for state in range(self.n_components_):
                    prob = self.transmat_[state][prev_state] * self.emissionprob_[state][word]
                    if prob > score:
                        score = prob
                        prev_state = state

            hidden_states.append(prev_state)

            log_likelihood += score
            # print(score, prev_state)

        return log_likelihood, hidden_states

    def _viterbi(self, X):
        log_likelihood, hidden_states = 0, []

        n_steps = len(X)

        if n_steps == 0:
            return log_likelihood, hidden_states
        
        m = np.zeros((self.n_components_, n_steps))
        parent = np.ones((self.n_components_, n_steps), dtype=int)

        for i, word in enumerate(X):
            if i == 0:
                for state in range(self.n_components_):
                    # print(self.startprob_[state], self.emissionprob_[state][word])
                    prob = self.startprob_[state] * self.emissionprob_[state][word]
                    if prob > m[state, i]:
                        parent[state, i] = state
                        m[state, i] = prob
            else:
                for s1 in range(self.n_components_):        # prev state
                    for s2 in range(self.n_components_):    # cur  state
                        # print(self.transmat_[s2, s1], self.emissionprob_[s2, word], m[s1, i-1])
                        prob = self.transmat_[s2, s1] * self.emissionprob_[s2, word] * m[s1, i-1]

                        if prob > m[s2, i]:
                            parent[s2, i] = s1
                            m[s2, i] = prob

        # print()
        # for x in m:
        #     print(x)
        # print()
        # for x in parent:
        #     print(x)

        mostLikelyStateIdx = np.argmax(m[:, -1])
        hidden_states.append(mostLikelyStateIdx)
        log_likelihood += m[mostLikelyStateIdx, -1]
        i = n_steps - 2

        if n_steps > 2:
            while i > 0:
                # put parent of state i
                hidden_states.append(parent[mostLikelyStateIdx, i+1])
                # add likelihood of state i
                log_likelihood += m[parent[mostLikelyStateIdx, i+1], i]
                mostLikelyStateIdx = hidden_states[-1]
                i -= 1
        if n_steps > 1:
            hidden_states.append(parent[mostLikelyStateIdx, 1])
            log_likelihood += m[hidden_states[-1], 0]



        return log_likelihood, reversed(hidden_states)

    def decode(self, X):

        if self.strategy == "viterbi":
            return self._viterbi(X)
        elif self.strategy == "greedy":
            return self._greedy(X)

In [3]:
dataset = load_dataset("lhoestq/conll2003")
dataset.save_to_disk("conll2003")

dataset_infos.json: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/281k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14041 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3250 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [24]:
random_state = 42

# nerTags = dataset["train"][:100]['ner_tags']
nerTags = dataset["train"][:]['ner_tags']
# tokens = dataset["train"][:100]['tokens']
tokens = dataset["train"][:]['tokens']
states = ["Other", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-MISC", "I-MISC", "UNK"]
observations = set()
maxLen = 0
observations.add(" ")
for token in tokens:
    maxLen = max(maxLen, len(token))
    for word in token:
        observations.add(word.lower())
observations = list(sorted(observations))
# print("Observations:", observations)

le = LabelEncoder()
encoded_data = le.fit_transform(observations)
# print("Encoded data:", encoded_data)

X = []
for token in tokens:
    encoded_tokens = le.transform([word.lower() for word in token])
    X.append(encoded_tokens.tolist())

# print(X)
# print(tokens)
# print(nerTags)

n_components = len(states)
n_observations = len(observations)
# print(n_components)
# print(n_observations)

model = HMMCustom(n_components=n_components, n_observations=n_observations)

model.fit(X, nerTags)

In [25]:
def insert_zeros_at_indices_numpy(original_list, excluded_indices):
    arr = np.array(original_list)
    excluded_indices = np.array(excluded_indices)
    # print(arr)
    # print(excluded_indices)
    new_size = len(excluded_indices) + len(arr)
    mask = np.ones(new_size, dtype=bool)
    mask[excluded_indices] = False
    # why [0] -> (array([3, 4]),)
    desired_indices = np.where(mask)[0]
    
    output = np.zeros(new_size, dtype=np.int8) + (n_components - 1)
    
    output[desired_indices] = arr
    # print(output)
    # print()
    return output.tolist()

In [26]:
n_samples = 10


# y_val = dataset["validation"][:n_samples]['ner_tags']
y_val = dataset["validation"][:]['ner_tags']
# X_val_tmp = dataset["validation"][:n_samples]['tokens']
X_val_tmp = dataset["validation"][:]['tokens']

# print("y val:", y_val)

X_val = []
dropped_words = {}
for i, token in enumerate(X_val_tmp):
    lowercaseToken = []
    for j, word in enumerate(token):
        word = word.lower()
        if word in le.classes_:
            lowercaseToken.append(word)
        else:
            if i not in dropped_words:
                dropped_words[i] = [(j, word)]
            else:
                dropped_words[i].append((j, word))
    encoded_tokens = le.transform(lowercaseToken)
    X_val.append(encoded_tokens.tolist())
    
# print("X val:", X_val)
# print("Dropped words:")
# for key, val in dropped_words.items():
#     for i, x in enumerate(val):
#         print(f"token {key+1}, word {i+1}: {x}")
# print()

y_test = []
for token in y_val:
    y_test.extend(token)
y_pred = []

for i, sample in enumerate(X_val):
    log_likelihood, hidden_states = model.decode(sample)
    hidden_states = list(hidden_states)
    # print(hidden_states)
    indices = []
    if i in dropped_words:
        for val in dropped_words[i]:
            indices.append(val[0])
        hidden_states = insert_zeros_at_indices_numpy(hidden_states, indices)
    # print(hidden_states)
    y_pred.extend(hidden_states)
    # print("Sample", i+1)
    # print("Log-likelihood of observations:", log_likelihood)
    # print("Most likely sequence:", [states[s] for s in hidden_states])
    # print("Observation sequence:", [states[s] for s in y_val[i]])
    
    # print()

In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score
# y_test = y_test[:772]
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Precision: {precision:.8f}")
print(f"Recall: {recall:.8f}")
print(f"F1-score: {f1:.8f}")

Precision: 0.71808139
Recall: 0.47966173
F1-score: 0.56280290


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, digits=4))

In [28]:
n_samples = 5


y_val = dataset["test"][:]['ner_tags']
X_val_tmp = dataset["test"][:]['tokens']

# print("y val:", y_val)

X_val = []
dropped_words = {}
for i, token in enumerate(X_val_tmp):
    lowercaseToken = []
    for j, word in enumerate(token):
        word = word.lower()
        if word in le.classes_:
            lowercaseToken.append(word)
        else:
            if i not in dropped_words:
                dropped_words[i] = [(j, word)]
            else:
                dropped_words[i].append((j, word))
    encoded_tokens = le.transform(lowercaseToken)
    X_val.append(encoded_tokens.tolist())
    
# print("X val:", X_val)
# print("Dropped words:")
# for key, val in dropped_words.items():
#     for i, x in enumerate(val):
#         print(f"token {key+1}, word {i+1}: {x}")
# print()

y_test = []
for token in y_val:
    y_test.extend(token)
y_pred = []

for i, sample in enumerate(X_val):
    log_likelihood, hidden_states = model.decode(sample)
    hidden_states = list(hidden_states)
    # print(hidden_states)
    indices = []
    if i in dropped_words:
        for val in dropped_words[i]:
            indices.append(val[0])
        hidden_states = insert_zeros_at_indices_numpy(hidden_states, indices)
    # print(hidden_states)
    y_pred.extend(hidden_states)
    # print("Sample", i+1)
    # print("Log-likelihood of observations:", log_likelihood)
    # print("Most likely sequence:", [states[s] for s in hidden_states])
    # print("Observation sequence:", [states[s] for s in y_val[i]])
    
    # print()

In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Precision: 0.6781
Recall: 0.4191
F1-score: 0.4954


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.9470    0.9168    0.9316     38323
           1     0.9239    0.2777    0.4270      1617
           2     0.8602    0.2396    0.3748      1156
           3     0.7644    0.3125    0.4436      1661
           4     0.6769    0.3437    0.4559       835
           5     0.8332    0.7458    0.7871      1668
           6     0.6221    0.5253    0.5696       257
           7     0.6471    0.4544    0.5339       702
           8     0.5062    0.3750    0.4309       216
           9     0.0000    0.0000    0.0000         0

    accuracy                         0.8279     46435
   macro avg     0.6781    0.4191    0.4954     46435
weighted avg     0.9201    0.8279    0.8586     46435



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
